In [6]:
import pandas as pd
import numpy as np

In [7]:
data=pd.read_csv("/Projects/dsin-python/DataSet/iris.csv",header=0)
data.sample(100)

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
3,4,4.6,3.1,1.5,0.2,setosa
75,76,6.6,3.0,4.4,1.4,versicolor
76,77,6.8,2.8,4.8,1.4,versicolor
30,31,4.8,3.1,1.6,0.2,setosa
88,89,5.6,3.0,4.1,1.3,versicolor
...,...,...,...,...,...,...
124,125,6.7,3.3,5.7,2.1,virginica
33,34,5.5,4.2,1.4,0.2,setosa
136,137,6.3,3.4,5.6,2.4,virginica
16,17,5.4,3.9,1.3,0.4,setosa


In [8]:
data["Species"]=data["Species"].map({"versicolor":0,"setosa":1,"virginica":2})
data.sample(100)

,Unnamed: 0,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
53,54,5.5,2.3,4.0,1.3,0
129,130,7.2,3.0,5.8,1.6,2
130,131,7.4,2.8,6.1,1.9,2
14,15,5.8,4.0,1.2,0.2,1
100,101,6.3,3.3,6.0,2.5,2
...,...,...,...,...,...,...
78,79,6.0,2.9,4.5,1.5,0
116,117,6.5,3.0,5.5,1.8,2
49,50,5.0,3.3,1.4,0.2,1
48,49,5.3,3.7,1.5,0.2,1


In [13]:
class KNN:
    def __init__(self,k):#初始化方法，k是整型表示邻居的个数
        self.k=k
    def fit(self,X,y):
        '''训练方法，只有在预测的时候才会调用，X是用于预测的类似数组、向量，[样本数量，样本特征]
        y是样本数量'''
        self.X=np.array(X)
        self.y=np.array(y)
    def predict(self,X):
        #X是用于预测的类似数组、向量，[样本数量，样本特征]
        #返回预测的结果
        X=np.array(X)
        result=[]
        for x in X:
            dis=np.sqrt(np.sum((x-self.X)**2,axis=1))       #按照行的方向求和 
            '''
            ndarray的运算，可以直接加减，每一位上的元素加减，避免了一个循环；
            如果是低维减多维，就先把低维扩张成维数相同的向量
            '''
            #返回数组排序后，每个元素在原数组中排序前的索引
            index=dis.argsort()
            #取距离最近的k个元素
            index=index[:self.k]
            #返回数组中每个非负整数出现的次数
            count=np.bincount(self.y[index])
            #返回值最大的元素，对应的索引，该索引就是我们判定的类别
            result.append(count.argmax())
        return np.asarray(result)

In [16]:
#对数据集做一个拆分，拆出测试集和训练集
#要保证数据集选择的随机性和完整性
t0=data[data["Species"]==0]
t1=data[data["Species"]==1]
t2=data[data["Species"]==2]
t0=t0.sample(len(t0),random_state=0)    #random_state用于保存打乱顺序以复盘
t1=t1.sample(len(t1),random_state=0)
t2=t2.sample(len(t2),random_state=0)
train_X=pd.concat([t0.iloc[:40,:-1],t1.iloc[:40,:-1],t2.iloc[:40,:-1]],axis=0)    #:40表示前40行，:-1表示直到倒数第一列
train_y=pd.concat([t0.iloc[:40,-1],t1.iloc[:40,-1],t2.iloc[:40,-1]],axis=0)
test_X=pd.concat([t0.iloc[40:,:-1],t1.iloc[40:,:-1],t2.iloc[40:,:-1]],axis=0)
test_y=pd.concat([t0.iloc[40:,-1],t1.iloc[40:,-1],t2.iloc[40:,-1]],axis=0)
#创建KNN对象，进行训练与测试
knn=KNN(k=3)
#训练
knn.fit(train_X,train_y)
#进行测试，获得测试结果
result=knn.predict(test_X)
#display(result)
#display(test_y)
display(np.sum(result==test_y))
display(len(result))

30

30